
# Final Project — News AI Platform (Capstone)

**Course:** ITAI 2373 – NLP  
**Submitter:** Esther Odaibo  
**Group:** Team004 (solo)  
**Folder:** `ITAI2373-NewsAI-Final`

This notebook extends the Mid‑Term NewsBot into a **production‑ready news analysis platform** with:
- **Advanced Content Analysis Engine**: improved classification, topic modeling, robust sentiment
- **Language Understanding & Generation**: extractive summarization; semantic search
- **Multilingual Intelligence**: language detection and translation for cross‑language analysis
- **Conversational Interface**: a simple **Gradio** app for natural language queries

> Run on **Google Colab (Free Tier)**. Keep dataset ≤ 2,000 rows.


## 0) Setup & Installs

In [ ]:
# Core libs
!pip -q install --upgrade pip
!pip -q install pandas numpy scikit-learn spacy nltk matplotlib nrclex textblob langdetect deep-translator gradio==4.44.0
!python -m spacy download en_core_web_sm -q

import nltk
nltk.download('punkt'); nltk.download('stopwords'); nltk.download('vader_lexicon')
print("✅ Setup complete.")



## 1) Data Ingestion (Kaggle or Manual Upload)

- Use the same approved dataset as midterm (e.g., **BBC**), or any instructor‑approved dataset.
- Options:
  1. **Kaggle API** (upload `kaggle.json` → download dataset)
  2. **Manual Upload** (CSV upload via Colab sidebar)


In [ ]:
from google.colab import files
import os, pandas as pd

def setup_kaggle():
    print("Upload kaggle.json")
    up = files.upload()
    assert 'kaggle.json' in up, "kaggle.json required"
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print("✅ Kaggle set")

def dl_bbc():
    print("Downloading BBC (learn-ai-bbc)")
    !kaggle competitions download -c learn-ai-bbc -q
    !unzip -o learn-ai-bbc.zip -d ./bbc >/dev/null
    print("🔎 Contents:")
    !ls -la ./bbc

def upload_csv():
    print("Upload your CSV now")
    files.upload()

print("Helpers ready: setup_kaggle(), dl_bbc(), upload_csv()")


## 2) Load & Prepare Dataset

In [ ]:
import os, re, pandas as pd, numpy as np

# Try to find a likely CSV (BBC or uploaded). Change if needed.
candidates = ["bbc/train.csv", "bbc/bbc-text.csv", "train.csv", "news.csv", "dataset.csv"]
csv_path = None
for p in candidates:
    if os.path.exists(p):
        csv_path = p; break

if csv_path is None:
    csv_path = "YOUR_UPLOADED_FILE.csv"  # <- set if using manual upload

print("Using CSV:", csv_path)

# Load with fallback encoding
try:
    df = pd.read_csv(csv_path)
except Exception as e:
    print("Default encoding failed:", e)
    df = pd.read_csv(csv_path, encoding="latin-1")

print("Shape:", df.shape); print("Columns:", df.columns.tolist()); print(df.head(2))

# Infer columns
possible_text = ['text','content','description','article','body','headline']
possible_cat  = ['category','label','class','topic','section']

def find_col(cols, cands):
    for c in cands:
        if c in cols: return c
    return None

text_col = find_col(df.columns, possible_text)
cat_col  = find_col(df.columns, possible_cat)
if text_col is None or cat_col is None:
    raise ValueError("Set text_col and cat_col to match your dataset.")

# Clean basic
df = df.dropna(subset=[text_col, cat_col]).drop_duplicates(subset=[text_col])

# Trim to ≤ 2000 rows for Colab
if len(df) > 2000:
    df = df.sample(2000, random_state=42)

df = df.rename(columns={text_col:'content', cat_col:'category'})[['content','category']].reset_index(drop=True)
df.head()


## 3) Multilingual Pipeline (Detect & Translate)

In [ ]:
from langdetect import detect
from deep_translator import GoogleTranslator

def detect_lang(text):
    try:
        return detect(text)
    except:
        return "unknown"

def to_english(text, src_lang=None):
    try:
        if not src_lang:
            src_lang = detect_lang(text)
        if src_lang != 'en' and src_lang != 'unknown':
            return GoogleTranslator(source=src_lang, target='en').translate(text)
        return text
    except:
        return text  # fallback

# Create English column for downstream NLP
df['lang'] = df['content'].astype(str).apply(detect_lang)
df['content_en'] = df.apply(lambda r: to_english(r['content'], r['lang']), axis=1)
df[['content','lang','content_en']].head(3)


## 4) Preprocessing (Clean → Lemmatize)

In [ ]:
import re, string, spacy
from nltk.corpus import stopwords

nlp = spacy.load("en_core_web_sm", disable=["ner"])
STOP = set(stopwords.words('english'))
url_re = re.compile(r'https?://\S+|www\.\S+')
html_re = re.compile(r'<.*?>')
num_re = re.compile(r'\b\d+\b')

def clean_text(t):
    t = str(t)
    t = url_re.sub(' ', t)
    t = html_re.sub(' ', t)
    t = t.translate(str.maketrans('', '', string.punctuation))
    t = num_re.sub(' ', t)
    t = re.sub(r'\s+', ' ', t).strip().lower()
    return t

def lemmatize(t):
    doc = nlp(t)
    out = [tok.lemma_ for tok in doc if not tok.is_stop and not tok.is_space]
    return " ".join(out)

df['clean'] = df['content_en'].apply(clean_text)
df['lemma'] = df['clean'].apply(lemmatize)
df[['content_en','lemma','category']].head(2)


## 5) Advanced Content Analysis Engine

In [ ]:
# Models: TF-IDF + (LinearSVC, LogReg, MultinomialNB) with evaluation
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
import pandas as pd

X_train, X_test, y_train, y_test = train_test_split(df['lemma'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

pipes = {
    "LinearSVC": Pipeline([("tfidf", TfidfVectorizer(max_features=25000, ngram_range=(1,2), min_df=2)),
                           ("clf", LinearSVC())]),
    "LogReg": Pipeline([("tfidf", TfidfVectorizer(max_features=25000, ngram_range=(1,2), min_df=2)),
                        ("clf", LogisticRegression(max_iter=300, solver='saga', C=2.0))]),
    "MNB": Pipeline([("tfidf", TfidfVectorizer(max_features=25000, ngram_range=(1,2), min_df=2)),
                     ("clf", MultinomialNB())])
}

scores = []
best = None; best_f1 = -1
for name, pipe in pipes.items():
    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, pred)
    f1 = f1_score(y_test, pred, average='macro')
    print(f"\n=== {name} ===")
    print("Accuracy:", round(acc,4), " Macro-F1:", round(f1,4))
    print(classification_report(y_test, pred, zero_division=0))
    scores.append((name, acc, f1))
    if f1 > best_f1:
        best_f1, best = f1, (name, pipe)

res_df = pd.DataFrame(scores, columns=['Model','Accuracy','MacroF1']).sort_values('MacroF1', ascending=False)
print("\nModel comparison:"); display(res_df)
best_name, best_pipe = best
print("Best model:", best_name)


### Topic Modeling (LDA on TF)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

cv = CountVectorizer(max_df=0.9, min_df=5, stop_words='english', max_features=8000)
X_counts = cv.fit_transform(df['clean'])

lda = LatentDirichletAllocation(n_components=6, learning_method='batch', random_state=42, max_iter=10)
lda.fit(X_counts)

terms = np.array(cv.get_feature_names_out())

def print_topics(model, terms, topn=12):
    for idx, comp in enumerate(model.components_):
        top_idx = comp.argsort()[-topn:][::-1]
        print(f"\nTopic {idx+1}:")
        print(", ".join(terms[top_idx]))

print_topics(lda, terms, topn=12)


## 6) Sentiment (VADER) & Summarization (TextRank-style)

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
from collections import Counter
import re

sia = SentimentIntensityAnalyzer()
df['sentiment_compound'] = df['content_en'].apply(lambda x: sia.polarity_scores(x)['compound'])

def compound_label(c):
    if c >= 0.05: return "positive"
    if c <= -0.05: return "negative"
    return "neutral"

df['sentiment_label'] = df['sentiment_compound'].apply(compound_label)
print(df['sentiment_label'].value_counts())

# Simple Extractive Summarizer (TextRank-like heuristic using TF-IDF weights)
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def summarize(text, max_sent=3):
    sents = re.split(r'(?<=[.!?])\s+', text.strip())
    if len(sents) <= max_sent:
        return text
    vec = TfidfVectorizer().fit(sents)
    X = vec.transform(sents)
    scores = (X.sum(axis=1)).A.ravel()
    topk = np.argsort(scores)[-max_sent:][::-1]
    return " ".join([sents[i] for i in sorted(topk)])

df['summary_en'] = df['content_en'].apply(lambda t: summarize(t, max_sent=3))
df[['content_en','summary_en']].head(2)


## 7) Semantic Search (TF‑IDF cosine)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

sem_vec = TfidfVectorizer(max_features=25000).fit(df['lemma'])
sem_mat = sem_vec.transform(df['lemma'])

def semantic_search(query, k=5):
    q_lemma = query
    # Preprocess like training
    q_lemma = lemmatize(clean_text(to_english(q_lemma)))
    q_vec = sem_vec.transform([q_lemma])
    sims = cosine_similarity(q_vec, sem_mat).ravel()
    idx = sims.argsort()[-k:][::-1]
    return df.iloc[idx][['content','category']].assign(score=sims[idx]).reset_index(drop=True)

semantic_search("central bank raises interest rates", k=3)


## 8) NER (spaCy)

In [ ]:
import spacy
nlp_ner = spacy.load("en_core_web_sm")

def extract_ents(text, limit=30):
    doc = nlp_ner(text)
    return [(e.text, e.label_) for e in doc.ents][:limit]

print(extract_ents(df['content_en'].iloc[0]))


## 9) Conversational Interface (Gradio App)

In [ ]:
import gradio as gr
import numpy as np

def pipeline_infer(text):
    # Multilingual normalize
    lang = detect_lang(text)
    text_en = to_english(text, lang)
    # Clean/lemma
    t = lemmatize(clean_text(text_en))
    # Classify
    pred = best_pipe.predict([t])[0]
    # Sentiment
    sent = sia.polarity_scores(text_en)['compound']
    sent_lbl = compound_label(sent)
    # Summary
    summ = summarize(text_en, max_sent=3)
    # NER
    ents = extract_ents(text_en)
    # Semantic search
    related = semantic_search(text_en, k=3).to_dict(orient='records')
    return {
        "language": lang,
        "predicted_category": pred,
        "sentiment": {"compound": float(sent), "label": sent_lbl},
        "summary_en": summ,
        "entities": ents,
        "related": related
    }

with gr.Blocks() as demo:
    gr.Markdown("# News AI Platform — Conversational Explorer")
    inp = gr.Textbox(label="Paste a news article or query")
    out = gr.JSON(label="Analysis")
    btn = gr.Button("Analyze")
    btn.click(pipeline_infer, inputs=inp, outputs=out)

# Uncomment to launch in Colab (interactive)
# demo.launch(share=False)  # set share=True if you want a public URL temporarily


## 10) Save Artifacts

In [ ]:
import joblib, os
os.makedirs("artifacts", exist_ok=True)
joblib.dump(best_pipe, "artifacts/final_best_model.joblib")
df[['content','category','lang']].head(10).to_csv("artifacts/sample_preview.csv", index=False)
print("✅ Saved artifacts.")


---
### Submission Details (Pre-filled)
Updated.